## Timeseries demo
<br>
There's two ways to extract a time series: for a point or for a polygon. There's an example for both below. <br>
If you prefer a GUI, then check out [https://viewer.terrascope.be/], and go to <b>Area of Interest</b>. <br>
When you use this Notebook, you have access to more layers than in the GUI.


In [49]:
import datetime
import requests
import matplotlib.pyplot as plt
import numpy as np
tsvBaseURL='https://services.terrascope.be/timeseries/v1.0/ts/'

response = requests.get(tsvBaseURL) #this returns the layers that are available



In [50]:
if response.status_code == 200:
    layerlist = response.json()['layers']
else:
    raise IOError(response.text)


let's investigate the layers that are offered. The list is not sorted, so let's sort it first.

In [51]:
layersSorted = sorted(layerlist, key=lambda k: k['name']) 

In [52]:
count=0
for l in layersSorted:
    if len(l['dates'])>0:
        count=count+1
        print(l['name'].ljust(40), '{0:4d} '.format(len(l['dates'])), min(l['dates'])[:19], 'to', max(l['dates'])[:19])
print(">>>", count, 'layers with data')
print('Layers with no dates')
count=0
for l in layersSorted:
    if len(l['dates'])==0:
        count=count+1
        print(l['name'])
print(">>>", count, 'layers without data')


BIOPAR_FAPAR300_V1_GLOBAL                 129  2013-10-16T00:00:00 to 2017-05-04T00:00:00
CGLS_LC100_COV_LCCS                      1598  2015-07-04T00:00:00 to 2020-08-08T00:00:00
CHIRPS_RAINFALL                           792  1998-01-11T00:00:00 to 2020-07-21T00:00:00
DCS4COP_CHL_V103                          799  2016-01-02T10:54:42 to 2020-05-30T12:27:01
DCS4COP_CLOUDCOVER_V103                   810  2016-01-02T10:54:42 to 2020-05-30T12:27:01
DCS4COP_FLANDERS_CHL                      454  2017-01-02T11:14:42 to 2018-10-31T11:02:01
DCS4COP_FLANDERS_CHL_V100                 750  2017-01-02T11:14:42 to 2020-02-10T10:52:01
DCS4COP_FLANDERS_CHL_V102                 890  2017-01-02T11:14:42 to 2020-06-02T10:56:31
DCS4COP_FLANDERS_CLOUDCOVER               454  2017-01-02T11:14:42 to 2018-10-31T11:02:01
DCS4COP_FLANDERS_CLOUDCOVER_L8            159  2017-01-03T10:40:38 to 2018-08-27T10:33:22
DCS4COP_FLANDERS_CLOUDCOVER_L8_V100        38  2019-05-10T10:30:59 to 2019-05-31T10:53:42
DCS4COP_FL

Let's first extract the timeseries of a point, for all layers that have 'TERRASCOPE' in their name

In [53]:
TSlayers = []
for l in layersSorted:
    if 'TERRASCOPE' in l['name']:
        TSlayers.append(l['name'])
TSlayers

['TERRASCOPE_S1_SLC_COHERENCE_V1_VH',
 'TERRASCOPE_S1_SLC_COHERENCE_V1_VV',
 'TERRASCOPE_S2_FAPAR_V2',
 'TERRASCOPE_S2_FCOVER_V2',
 'TERRASCOPE_S2_LAI_V2',
 'TERRASCOPE_S2_NDVI_V2']

In [54]:
def getTimeseriesForPoint(covId,tsvBaseURL='https://services.terrascope.be/timeseries/v1.0/ts/',
                          start=datetime.date(2016,1,1),end=datetime.date(2030,12,31),
                          lat=51.146,lon=3.682,printURL=False):

    tsURL = tsvBaseURL + covId + '/point'
    payload = {
            'lon': str(lon),
            'lat': str(lat),
            'startDate': start.strftime('%Y-%m-%d'),
            'endDate': end.strftime('%Y-%m-%d')
    }
    if printURL:
        print(tsURL,payload)
    response=requests.get(tsURL,params=payload)
    if response.status_code == 200:
        timeseries = response.json()['results']
        return(timeseries)
    else:
        return([])
    
    

In [55]:
print('layer'.ljust(40), 'points')
print('---------------------------------------  ------')
for l in TSlayers:
    ts = getTimeseriesForPoint(covId=l,start=datetime.date(2020,1,1), end = datetime.date(2020,7,1), printURL=False)
    print(l.ljust(40),'{0:6d}'.format(len(ts)))


layer                                    points
---------------------------------------  ------
TERRASCOPE_S1_SLC_COHERENCE_V1_VH             0
TERRASCOPE_S1_SLC_COHERENCE_V1_VV             0
TERRASCOPE_S2_FAPAR_V2                       97
TERRASCOPE_S2_FCOVER_V2                      97
TERRASCOPE_S2_LAI_V2                         97
TERRASCOPE_S2_NDVI_V2                        97


So, there are no results for coherence. This needs further analysis.<br>
<br>
Let's have a look at the last time series that we have extracted, for NDVI.<br>
We'll only list the points that are valid (all others cannot be determined due to cloud cover.

In [56]:
print('date        average')
print('----------  -------')
for d in ts:
    if d['result']['validCount']>0:
        print(d['date'], '  {0:.4f}'.format(d['result']['average']))

date        average
----------  -------
2020-01-01   0.2360
2020-01-04   0.3240
2020-01-06   0.3200
2020-01-19   0.3640
2020-01-26   0.3320
2020-02-13   0.4080
2020-03-19   0.4880
2020-03-21   0.5160
2020-03-24   0.5080
2020-03-26   0.5080
2020-03-31   0.5000
2020-04-05   0.5600
2020-04-08   0.6000
2020-04-10   0.5960
2020-04-13   0.7040
2020-04-20   0.7760
2020-04-23   0.7960
2020-04-25   0.7840
2020-05-05   0.8800
2020-05-15   0.8760
2020-05-18   0.8800
2020-05-20   0.8280
2020-05-23   0.8520
2020-05-25   0.8800
2020-05-28   0.9160
2020-05-30   0.8800
2020-06-07   0.8480
2020-06-22   0.6080
2020-06-24   0.8760


On to time series for a polygon then. <br>
This is done using a POST request, rather than a GET request for a point.

In [57]:
def getTimeseriesForPolygon(covId, polylist,
                            tsvBaseURL='https://services.terrascope.be/timeseries/v1.0/ts/',
                            start=datetime.date(2016,1,1),end=datetime.date(2030,12,31),
                            printURL=False):


    tsURL = tsvBaseURL + covId 
    # print(tsvURL)
    tsURL = tsURL +'/geometry'

    payload = {
            "type": "Feature",
            "geometry": {
            "type": "Polygon",
            "coordinates": [
            
                polylist
            
            ]
            }
        }


    payload['startDate'] = start.strftime('%Y-%m-%d')
    payload['endDate'] = end.strftime('%Y-%m-%d')
    if printURL:
        print(tsURL, payload)
    response=requests.post(url=tsURL,json=payload)

    if response.status_code==200:
        timeSeries = response.json()['results']   
        return(timeSeries)
    else:
        return([])

In [58]:
polygon=[]

point = [3.65, 51.14]
polygon.append(point)
point = [3.66, 51.14]
polygon.append(point)
point = [3.66, 51.15]
polygon.append(point)
point = [3.65, 51.15]
polygon.append(point)
point = [3.65, 51.14]
polygon.append(point)


In [59]:
PolyResults=[]
for l in TSlayers:
    Tresult={}
    Tresult['name']=l
    Tresult['series']= getTimeseriesForPolygon(covId=l,polylist=polygon, start = datetime.date(2020,1,1), end = datetime.date(2020,5,1),printURL=False)
    PolyResults.append(Tresult)
    print(Tresult['name'], len(Tresult['series']))
    

TERRASCOPE_S1_SLC_COHERENCE_V1_VH 1036
TERRASCOPE_S1_SLC_COHERENCE_V1_VV 1036
TERRASCOPE_S2_FAPAR_V2 839
TERRASCOPE_S2_FCOVER_V2 839
TERRASCOPE_S2_LAI_V2 839
TERRASCOPE_S2_NDVI_V2 839


The start and end date are not taken into account, So, we'll print just the start of the time series.<Br>

In [60]:
ind=3
print(PolyResults[ind]['name'])
print('date'.ljust(11), 'total', ' valid', '  ratio','average')
print('---------- ------ ------  ------ -------')
cutoff=0.9
count=0
maxcount=25
for r in PolyResults[ind]['series']:
    if r['result']['validCount']/r['result']['totalCount'] > cutoff and count<maxcount:
        print(r['date'], '{0:6d}'.format(r['result']['totalCount']), '{0:6d}'.format(r['result']['validCount']), 
              ' {0:.4f}'.format(r['result']['validCount']/r['result']['totalCount']), ' {0:.4f}'.format(r['result']['average']))
        count=count+1

TERRASCOPE_S2_FCOVER_V2
date        total  valid   ratio average
---------- ------ ------  ------ -------
2015-07-06  21576  20566  0.9532  0.5671
2015-07-16  21576  21576  1.0000  0.5454
2015-12-03  21576  21260  0.9854  0.3427
2016-01-25  21576  20752  0.9618  0.3933
2016-03-12  21576  21576  1.0000  0.4106
2016-03-15  21576  21563  0.9994  0.3901
2016-04-01  21576  21345  0.9893  0.4722
2016-04-04  21576  20997  0.9732  0.4566
2016-04-14  21576  20997  0.9732  0.3950
2016-05-01  21576  21227  0.9838  0.5496
2016-05-04  21576  21250  0.9849  0.5605
2016-05-11  21576  20492  0.9498  0.4479
2016-06-23  21576  21519  0.9974  0.5479
2016-07-03  21576  21461  0.9947  0.5571
2016-07-20  21576  21330  0.9886  0.5140
2016-09-08  21576  21340  0.9891  0.5041
2016-09-21  21576  21576  1.0000  0.4320
2016-09-28  21576  21548  0.9987  0.3785
2016-10-01  21576  20247  0.9384  0.3068
2016-10-08  21576  19955  0.9249  0.3534
2016-10-11  21576  21084  0.9772  0.2092
2016-10-21  21576  19645  0.9105 

This is okay: 
- the total number is the number of pixels in the polygon; it is constant, because the polygon doesn't change.
- the valid number is the number of pixels that are not covered by clouds. We are just printing the measures that have 90% good pixels
- the ratio is valid/total
- average is the value of the indicator at that date averaged over all pixels
<br>
No, let's have a look at the coherence time series

In [61]:
ind=1
print(PolyResults[ind]['name'])
print('date'.ljust(11), 'total', ' valid', '  ratio','average')
print('---------- ------ ------  ------ -------')
cutoff=0.9
count=0
maxcount=25
for r in PolyResults[ind]['series']:
    if r['result']['validCount']/r['result']['totalCount'] > cutoff and count<maxcount:
        print(r['date'], '{0:6d}'.format(r['result']['totalCount']), '{0:6d}'.format(r['result']['validCount']), 
              ' {0:.4f}'.format(r['result']['validCount']/r['result']['totalCount']), ' {0:.4f}'.format(r['result']['average']))
        count=count+1

TERRASCOPE_S1_SLC_COHERENCE_V1_VV
date        total  valid   ratio average
---------- ------ ------  ------ -------
2016-09-26  64728  64728  1.0000  0.4301
2016-09-29 107880 107880  1.0000  0.3240
2016-10-02  64728  64728  1.0000  0.3393
2016-10-04 172608 172520  0.9995  0.4023
2016-10-05 107880 107880  1.0000  0.3902
2016-10-08 107880 107880  1.0000  0.3969
2016-10-10  86304  86264  0.9995  0.3651
2016-10-11 107880 107880  1.0000  0.4649
2016-10-14 107880 107880  1.0000  0.3864
2016-10-16  86304  86304  1.0000  0.4627
2016-10-17 107880 107880  1.0000  0.3909
2016-10-20 107880 107880  1.0000  0.3239
2016-10-22  86304  85392  0.9894  0.4233
2016-10-23 107880 107880  1.0000  0.4000
2016-10-26  86304  86304  1.0000  0.3800
2016-10-28  86304  85644  0.9924  0.5507
2016-10-29  64728  64728  1.0000  0.5503
2016-11-01  86304  86304  1.0000  0.4130
2016-11-03  64728  64728  1.0000  0.3943
2016-11-04  64728  64728  1.0000  0.4701
2016-11-07 107880 107880  1.0000  0.4609
2016-11-09  64728  5964

This is strange: 
- the total number is the number of pixels in the polygon; it is <b>not</b> constant, probably because there are multiple entries per date in the time series.
- the valid number is <b>not</b> constant and equal to total. As RADAR is cloud penetrating, there's no reason for pixels to be invalid
